## Install libraries, modules

In [1]:
# !python -m spacy download es_core_news_sm

In [2]:
# !python -m spacy download en_core_web_sm

In [3]:
# pip install spacy

In [4]:
# pip install pandas

In [5]:
import pandas as pd

/Users/paigelee/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [6]:
import random

In [7]:
# install spacy lang models
import spacy

spaNLP = spacy.load("es_core_news_sm")
engNLP = spacy.load("en_core_web_sm")

## 1. Load raw texts

In [8]:
# with open('eng_garden.txt','r') as f:
#      engtext = f.read()
# with open('spa_garden.txt','r') as f:
#      spatext = f.read()

### Apply language model

In [123]:
# spadoc = spaNLP(spatext)
# engdoc = engNLP(engtext)

In [124]:
# # sentenize
# spasents = []
# engsents = []
# for sent in spadoc.sents:
#     spasents.append(sent.text)
# for sent in engdoc.sents:
#     engsents.append(sent.text)

## 2. Write standardized files (one line per sentence) for input to Bleualign

In [128]:
# # these are the inputs to bleualign
# with open('sourcetext.txt','w') as f:
#     f.write('\n'.join(spasents))
# with open('targettext.txt','w') as f:
#     f.write('\n'.join(engsents))

### At this point, I also generate sourcetexttranslation.text using Google Translate and run Bleualign on the texts on my terminal.

## [START HERE] 3. Read sentence-aligned files (from Bleualign)

In [9]:
import random

In [10]:
with open('sourceoutput.txt','r') as f:
    srcsents = f.read().split('\n')
with open('targetoutput.txt','r') as f:
    tgtsents = f.read().split('\n')

In [11]:
i = random.choice(range(len(srcsents)))
srcsents[i], tgtsents[i]

('Me pareció increíble que ese día sin premoniciones ni símbolos fuera el de mi muerte 2 implacable.',
 'It seemed incredible that this day, a day without warnings or omens, might be that of my implacable death.')

In [12]:
srcsents[129:132], tgtsents[129:132]

(['»-¡Un laberinto de marfil! -exclamé-.',
  'Un laberinto mínimo... »-Un laberinto de símbolos -corrigió-.',
  'Un invisible laberinto de tiempo.'],
 ['"An ivory labyrinth?" I exclaimed.',
  '"A symbolic labyrinth," he corrected me. "An invisible labyrinth of time.',
  '"An invisible labyrinth of time.'])

## 4. Parse aligned sentences using Astred word alignment

In [13]:
# !pip install astred[stanza]

In [14]:
# !pip install git+https://github.com/BramVanroy/awesome-align.git@astred_compat

In [15]:
import time
from astred import AlignedSentences, Sentence

## Running Astred Word Alignment (https://github.com/BramVanroy/astred/issues/3)

In [16]:
from astred.aligned import AlignedSentences, Sentence
from astred.aligner import Aligner
from astred.utils import load_parser

In [17]:
nlp_en = load_parser("en", "stanza", is_tokenized=False, verbose=True)
nlp_es = load_parser("es", "stanza", is_tokenized=False, verbose=True)
aligner = Aligner()

your_data = zip(srcsents, tgtsents)

alignmentList = []
i = 0
success = 0
for sent_es_str, sent_en_str in your_data:
    if i % 10 == 0:
        print(f'{i}/{len(srcsents)} sentences parsed.')
    try:
#         joinedsents = sent_en_str.replace('.',';')
#         if joinedsents[-1] == ';':
#             joinedsents = joinedsents[:-1] + '.'
#         sent_en = Sentence.from_text(joinedsents, nlp_en)
        sent_en = Sentence.from_text(sent_en_str, nlp_en)
        sent_es = Sentence.from_text(sent_es_str, nlp_es)
        aligned = AlignedSentences(sent_es, sent_en, aligner=aligner)
        alignmentList.append({
            'spanish_text' : sent_es_str,
            'english_text' : sent_en_str,
            'spanish_nlp' : sent_es,
            'english_nlp' : sent_en,
            'alignment' : aligned
        })
        success += 1
    except:
        alignmentList.append({
            'spanish_text' : sent_es_str,
            'english_text' : sent_en_str,
            'spanish_nlp' : 'Error',
            'english_nlp' : 'Error',
            'alignment' : 'Error'
        })
    i += 1
#     if i == 20:
#         break

2022-02-26 08:06:19 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-02-26 08:06:19 INFO: Use device: cpu
2022-02-26 08:06:19 INFO: Loading: tokenize
2022-02-26 08:06:19 INFO: Loading: pos
2022-02-26 08:06:21 INFO: Loading: lemma
2022-02-26 08:06:21 INFO: Loading: depparse
2022-02-26 08:06:23 INFO: Done loading processors!
2022-02-26 08:07:20 WARNING: Language es package default expects mwt, which has been added
2022-02-26 08:07:20 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |

2022-02-26 08:07:20 INFO: Use device: cpu
2022-02-26 08:07:20 INFO: Loading: tokenize
2022-02-26 08:07:20 INFO: Loading: mwt
2022-02-26 08:07:20 INFO: Loading: pos
2022-02-26 08:

0/219 sentences parsed.
10/219 sentences parsed.
20/219 sentences parsed.
30/219 sentences parsed.
40/219 sentences parsed.
50/219 sentences parsed.
60/219 sentences parsed.
70/219 sentences parsed.
80/219 sentences parsed.
90/219 sentences parsed.
100/219 sentences parsed.
110/219 sentences parsed.
120/219 sentences parsed.
130/219 sentences parsed.
140/219 sentences parsed.
150/219 sentences parsed.
160/219 sentences parsed.
170/219 sentences parsed.
180/219 sentences parsed.
190/219 sentences parsed.
200/219 sentences parsed.
210/219 sentences parsed.


## 5. Alignment done! View alignments below.

In [21]:
print(f'success rate of word alignment on sentences is {success}/{i} sents or {success/i}')

success rate of word alignment on sentences is 164/219 sents or 0.7488584474885844


### Sequence alignment AND word alignment (more importantly)

In [22]:
a = random.choice(alignmentList)
for pair in a['alignment'].aligned_seq_spans:
    print(pair[0].text, '\t\t'+pair[1].text)
print()
print()
for pair in a['alignment'].aligned_words:
    print(pair[0].text, '\t\t'+pair[1].text)

 		
 		as
 		as
 		possible
Aniquilado , trémulo , me encogí en la otra punta de el sillón , 		Shattered , trembling , I huddled in the distant corner of the seat ,
lejos 		far
de el temido cristal . 		from the fearful window .


[[NULL]] 		[[NULL]]
[[NULL]] 		as
[[NULL]] 		as
[[NULL]] 		possible
Aniquilado 		Shattered
, 		,
trémulo 		trembling
, 		,
me 		I
encogí 		huddled
en 		in
la 		the
otra 		distant
punta 		corner
de 		of
el 		the
sillón 		seat
, 		,
lejos 		far
de 		from
el 		the
temido 		fearful
cristal 		window
. 		.


In [23]:
### Write something not completely huge to the Pickle
pickleList = []
for a in alignmentList:
    if a['alignment'] != 'Error': 
        spanishTokens = [{
            'tokenid' : token.id, 
            'POS' : token.upos, 
            'text' : token.text, 
            'lemma' : token.lemma,
            'features' : token.feats
        } for token in a['spanish_nlp']]
        englishTokens = [{
            'tokenid' : token.id, 
            'POS' : token.upos, 
            'text' : token.text, 
            'lemma' : token.lemma,
            'features' : token.feats
        } for token in a['english_nlp']]
        alignmentTuples = [(pair[0].id, pair[1].id) for pair in a['alignment'].aligned_words]
    
    else:
        spanishTokens = [{
            'tokenid' : token.i, 
            'POS' : token.pos_, 
            'text' : token.text, 
            'lemma' : token.lemma_,
            'features' : 'Error'
        } for token in spaNLP(a['spanish_text'])]
        englishTokens = [{
            'tokenid' : token.i, 
            'POS' : token.pos_, 
            'text' : token.text, 
            'lemma' : token.lemma_,
            'features' : 'Error'
        } for token in engNLP(a['english_text'])]
        
        alignmentTuples = []
    
    # build list
    pickleList.append({
        'spanishRawText' : a['spanish_text'],
        'englishRawText' : a['english_text'],
        'spanishTokenList' : spanishTokens,
        'englishTokenList' : englishTokens,
        'alignmentTuples' : alignmentTuples
    })

In [32]:
p =random.choice(pickleList)
print(p['alignmentTuples'])

[]


In [25]:
import pickle

with open('alignment2-26.pickle', 'wb') as handle:
    pickle.dump(pickleList, handle, protocol=pickle.HIGHEST_PROTOCOL)

## ignore below

In [11]:
# jank version which worked ok but took SO long (8+ hrs)

# # alignment tool (takes a freaking long time)
# start = time.time()

# alignmentList = []
# success = 0
# total = 0
# for i in range(len(srcsents)):
#     if srcsents[i] == '' or tgtsents[i] == '':
#         alignmentList.append((srcsents[i],tgtsents[i]))
#         print(f'blank space on sentence {i}')
#     else:
#         try:
#             sent_sp = Sentence.from_text(srcsents[i], "es", is_tokenized=False)
#             sent_es = Sentence.from_text(tgtsents[i], "en", is_tokenized=False)
#             aligned = AlignedSentences(sent_sp, sent_es)
#             print(aligned.aligned_words)
#             alignmentList.append(aligned)
#             success += 1
#             print(f'successfully aligned sentence {i}')
#         except:
#             alignmentList.append((srcsents[i],tgtsents[i]))
#             print(f'spacy failure on sentence {i}')
#     total += 1
    
# end = time.time()
# print(f'only took {end-start} seconds or {(end-start)/60} minutes to run')